In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import scipy.stats as st

In [2]:
df_libor=pd.read_excel("/home/user/Desktop/env2/VAR/mvp_3/floating_rate_port.xlsx")

xls = pd.ExcelFile('/home/user/Desktop/env2/VAR/mvp_3/interest_rates.xlsx')
LIBOR_int=pd.read_excel(xls,"LIBOR")
LIBOR_int.index=LIBOR_int["Date"]

SOFR_int=pd.read_excel(xls,"SOFR")
SOFR_int.index=SOFR_int["Date"]

SONIA_int=pd.read_excel(xls,"SONIA")
SONIA_int.index=SONIA_int["Date"]

TIIE_int=pd.read_excel(xls,"TIIE")
TIIE_int.index=TIIE_int["Date"]

mapping_dict={"LIBOR":LIBOR_int,"SOFR":SOFR_int,"SONIA":SONIA_int,"TIIE":TIIE_int}
effective_col_calc={"Monthly":12,"Quarterly":4,"Six Months":2,"Annually":1}
date_interval={"Monthly":30,"Quarterly":90,"Six Months":180,"Annually":360}

In [3]:
df_libor_monthly=df_libor#[df_libor["Comp_Freq"]=="Monthly"]
df_libor_monthly.reset_index(inplace=True)
df_libor_monthly.drop("index",axis=1,inplace=True)
df_libor_monthly

,Loan ID,Notional_Amt,Loan_Nature,Start Date,Maturity Date,Comp_Freq,Duration,Base Rate,Rate Tenor,New ARR,Spread
0,1,10000000,Floating,2019-04-02,2019-06-30,Monthly,3,LIBOR,1M,SOFR,1.5
1,2,10000000,Floating,2019-04-02,2019-09-30,Monthly,6,LIBOR,1M,SOFR,1.5
2,3,10000000,Floating,2019-04-02,2020-03-31,Monthly,12,LIBOR,1M,SOFR,1.5
3,4,10000000,Floating,2019-04-02,2019-09-30,Quarterly,2,LIBOR,3M,SOFR,1.5
4,5,10000000,Floating,2019-04-02,2020-03-31,Quarterly,4,LIBOR,3M,SOFR,1.5
5,6,10000000,Floating,2019-04-02,2021-03-31,Quarterly,8,LIBOR,3M,SOFR,1.5
6,7,10000000,Floating,2019-04-02,2020-03-31,Six Months,2,LIBOR,6M,SOFR,1.5
7,8,10000000,Floating,2019-04-02,2021-03-31,Six Months,4,LIBOR,6M,SOFR,1.5
8,9,10000000,Floating,2019-04-02,2021-03-31,Six Months,4,LIBOR,6M,SOFR,1.5
9,10,10000000,Floating,2018-05-01,2020-03-31,Annually,2,LIBOR,12M,SOFR,1.5


In [5]:
inter_dict_base={}
inter_dict_ARR={}
inter_dict_ARR_adj={}
def get_interest_rate(int_df,start_date,tenor):
    interest_rate=int_df[int_df["Date"]==start_date][tenor].values
    return interest_rate
            
    
count=0
while(count<len(df_libor_monthly)):
    #rint(type(df_libor_monthly.loc[count,"Start Date"]))
    inter_dict_base[df_libor_monthly.loc[count,"Loan ID"]]=[]
    inter_dict_ARR[df_libor_monthly.loc[count,"Loan ID"]]=[]
    inter_dict_ARR_adj[df_libor_monthly.loc[count,"Loan ID"]]=[]
    interest_monthly=[]
    for j in range(0,df_libor_monthly.loc[count,"Duration"]):
        ####Calculation of interest with Base Rate######
        freq_type=df_libor_monthly.loc[count,"Comp_Freq"]
        start_date= df_libor_monthly.loc[count,"Start Date"]+timedelta(days=int(j)*date_interval[freq_type])
        int_df=mapping_dict[df_libor_monthly.loc[count,"Base Rate"]]
        tenor= df_libor_monthly.loc[count,"Rate Tenor"]
        interest_rate=get_interest_rate(int_df,start_date,tenor)
        while interest_rate.size==0:
            start_date= start_date+ timedelta(days=-1)
            #print(start_date)
            interest_rate=get_interest_rate(int_df,start_date,tenor)
            
        
        spread_value=df_libor_monthly.loc[count,"Spread"] 
        
        notional=df_libor_monthly.loc[count,"Notional_Amt"]
        effective_rate= ((1+(interest_rate[0]+spread_value)/100)**(1/effective_col_calc[freq_type]))-1
        inter_dict_base[df_libor_monthly.loc[count,"Loan ID"]].append(notional*effective_rate)
        
        ####Calculation of interest with ARR Rate######
        int_df=mapping_dict[df_libor_monthly.loc[count,"New ARR"]]
        interest_rate=get_interest_rate(int_df,start_date,tenor)
        while interest_rate.size==0:
            start_date= start_date+ timedelta(days=-1)
            #print(start_date)
            interest_rate=get_interest_rate(int_df,start_date,tenor)
            
        
        spread_value=df_libor_monthly.loc[count,"Spread"] 
        
        notional=df_libor_monthly.loc[count,"Notional_Amt"]
        effective_rate= ((1+(interest_rate[0]+spread_value)/100)**(1/effective_col_calc[freq_type]))-1
        inter_dict_ARR[df_libor_monthly.loc[count,"Loan ID"]].append(notional*effective_rate)
    df_libor_monthly.loc[count,"Base Rate Total Int"]= np.sum(inter_dict_base[df_libor_monthly.loc[count,"Loan ID"]])
    df_libor_monthly.loc[count,"ARR Total Int"]= np.sum(inter_dict_ARR[df_libor_monthly.loc[count,"Loan ID"]])
    
    ###weighted AVG Calculation###
    duration=df_libor_monthly.loc[count,"Duration"]
    df_libor_monthly.loc[count,"ARR Weighted AVG"]=(((((df_libor_monthly.loc[count,"ARR Total Int"]/notional)/duration)+1)**effective_col_calc[freq_type])-1)*100
    df_libor_monthly.loc[count,"ARR Adj Spread (%)"]=((((1+((df_libor_monthly.loc[count,"Base Rate Total Int"]/duration)/notional))**effective_col_calc[freq_type])-1)*100)-df_libor_monthly.loc[count,"ARR Weighted AVG"]
    Adjusted_spred=df_libor_monthly.loc[count,"ARR Adj Spread (%)"]
    
    ####Calculation of interest with ARR adjusted Rate######
    for j in range(0,duration):
        start_date= df_libor_monthly.loc[count,"Start Date"]+timedelta(days=int(j)*date_interval[freq_type])
        int_df=mapping_dict[df_libor_monthly.loc[count,"New ARR"]]
        interest_rate=get_interest_rate(int_df,start_date,tenor)
        while interest_rate.size==0:
            start_date= start_date+ timedelta(days=-1)
            #print(start_date)
            interest_rate=get_interest_rate(int_df,start_date,tenor)
            
        
        spread_value=df_libor_monthly.loc[count,"Spread"] 
        
        notional=df_libor_monthly.loc[count,"Notional_Amt"]
        effective_rate= ((1+(interest_rate[0]+spread_value+Adjusted_spred)/100)**(1/effective_col_calc[freq_type]))-1
        inter_dict_ARR_adj[df_libor_monthly.loc[count,"Loan ID"]].append(notional*effective_rate)
    
    df_libor_monthly.loc[count,"ARR Adjusted Spread Total Int"]= np.sum(inter_dict_ARR_adj[df_libor_monthly.loc[count,"Loan ID"]])
    count+=1
print(df_libor_monthly)
    
        
    
    

    Loan ID  Notional_Amt Loan_Nature Start Date Maturity Date   Comp_Freq  \
0         1      10000000    Floating 2019-04-02    2019-06-30     Monthly   
1         2      10000000    Floating 2019-04-02    2019-09-30     Monthly   
2         3      10000000    Floating 2019-04-02    2020-03-31     Monthly   
3         4      10000000    Floating 2019-04-02    2019-09-30   Quarterly   
4         5      10000000    Floating 2019-04-02    2020-03-31   Quarterly   
5         6      10000000    Floating 2019-04-02    2021-03-31   Quarterly   
6         7      10000000    Floating 2019-04-02    2020-03-31  Six Months   
7         8      10000000    Floating 2019-04-02    2021-03-31  Six Months   
8         9      10000000    Floating 2019-04-02    2021-03-31  Six Months   
9        10      10000000    Floating 2018-05-01    2020-03-31    Annually   
10       11      10000000    Floating 2018-05-01    2021-03-31    Annually   
11       12      10000000    Floating 2020-04-01    2020-06-30  